In [18]:
%local 
import yaml
from wl_monitor.wl_api import WienerLinienAPI
from wl_monitor.mqtt_client import get_mqtt_client
from wl_monitor.publisher import publish_departures, subscribe_departures

In [5]:
%local 

with open("../config/config.yaml") as f:
    config = yaml.safe_load(f)

stations = config["stations"]
mqtt_cfg = config["mqtt"]
sender = config.get("wienerlinien", {}).get("sender", "")
interval = config.get("interval", 30)


In [6]:
%local 

api = WienerLinienAPI(sender=sender)
client = get_mqtt_client(mqtt_cfg["address"], mqtt_cfg["port"], "publ")
client.loop_start()

Connected to MQTT broker with result code 0

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
%local
def extract_aufzugsinfo(response_json):
    infos = response_json.get("data", {}).get("trafficInfos", [])
    aufzug_infos = []

    for info in infos:
        if "aufzug" in info.get("title", "").lower() or \
           "aufzug" in info.get("description", "").lower() or \
           info.get("attributes", {}).get("status", "").lower() in ["außer betrieb", "geplante wartung"]:

            aufzug_infos.append({
                "title": info.get("title"),
                "status": info.get("attributes", {}).get("status"),
                "station": info.get("attributes", {}).get("station"),
                "location": info.get("attributes", {}).get("location"),
                "reason": info.get("attributes", {}).get("reason"),
            })

    return aufzug_infos
import requests
raw = requests.get("https://www.wienerlinien.at/ogd_realtime/monitor", params={
    "stopId": 4611,
    "activateTrafficInfo": ["aufzugsinfo"]
}).json()
print(raw)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.wienerlinien.at:443


{'data': {'monitors': [{'locationStop': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [16.3375343, 48.1908296]}, 'properties': {'name': '60200436', 'title': 'Gumpendorfer Straße', 'municipality': 'Wien', 'municipalityId': 49000001, 'type': 'stop', 'coordName': 'WGS84', 'gate': '2', 'attributes': {'rbl': 4611}}}, 'lines': [{'name': 'U6', 'towards': 'SIEBENHIRTEN', 'direction': 'R', 'platform': '2', 'richtungsId': '2', 'barrierFree': True, 'realtimeSupported': True, 'trafficjam': False, 'departures': {'departure': [{'departureTime': {'timePlanned': '2025-12-07T13:02:28.000+0100', 'timeReal': '2025-12-07T13:02:28.000+0100', 'countdown': 4}}, {'departureTime': {'timePlanned': '2025-12-07T13:07:28.000+0100', 'timeReal': '2025-12-07T13:07:28.000+0100', 'countdown': 9}}, {'departureTime': {'timePlanned': '2025-12-07T13:12:30.000+0100', 'countdown': 13}, 'vehicle': {'name': 'U6', 'towards': 'Siebenhirten', 'direction': 'R', 'platform': '2', 'richtungsId': '2', 'barrierFree':

DEBUG:urllib3.connectionpool:https://www.wienerlinien.at:443 "GET /ogd_realtime/monitor?stopId=4611&activateTrafficInfo=aufzugsinfo HTTP/1.1" 200 None


In [17]:
%local 
for station in stations:
    deps = api.fetch_departures(station["rbl"])
    publish_departures(client, station["name"], deps, mqtt_cfg["topic_prefix"])

Published message with mid 9
 New data for Karlsplatz:
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 0}
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 6}
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 8}


In [16]:
%local
def handle_departure_update_(data):
    print(f" New data for {data['stop']}:")
    for d in data["departures"]:
        print(" →", d)

subscribe_departures(client, stations[0]["name"], callback=handle_departure_update_)

📡 Subscribed to ubahn/Karlsplatz
 New data for Karlsplatz:
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 0}
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 6}
 → {'stop': 'Karlsplatz', 'line': 'U4', 'towards': 'HÜTTELDORF    ', 'minutes': 8}
